In [3]:
import sys
from os.path import join
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import torch as pt
sys.path.append("/home/spin_se/Software/flowtorch")
import flowtorch
from flowtorch.data import TAUSurfaceDataloader
from flowtorch import DATASETS

Refer to the installation instructions at https://github.com/FlowModelingControl/flowtorch
If you are not using the TecplotDataloader, ignore this warning.


In [4]:
path_ds = DATASETS["tau_surface_import"]
file_name = "tau_euler.para"
loader = TAUSurfaceDataloader(join(path_ds,file_name))

In [5]:
times = loader.write_times
times

['1.000e+00',
 '2.000e+00',
 '3.000e+00',
 '4.000e+00',
 '5.000e+00',
 '6.000e+00',
 '7.000e+00',
 '8.000e+00',
 '9.000e+00',
 '1.000e+01']

In [6]:
loader.zone_names

['1', '2', '3', '4', '5', '6', '7']

In [7]:
loader.vertices.shape[0]

NameError: name 'vertices' is not defined

In [ ]:
loader.field_names[times[0]]

In [ ]:
cp = loader.load_snapshot("cp", times)

In [ ]:
x = loader.vertices[:,0]
y = loader.vertices[:,1]
z = loader.vertices[:,2]

In [ ]:
cp[:,0].shape

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x,y,c=cp[:,0])
plt.show()

In [8]:
mfile = join(loader._para.path, loader._para.config["primary_grid"])
mdata = Dataset(mfile)
mdata.variables

{'marker': <class 'netCDF4._netCDF4.Variable'>
 int32 marker(no_of_markers)
 unlimited dimensions: 
 current shape = (7,)
 filling on, default _FillValue of -2147483647 used,
 'centaur_key': <class 'netCDF4._netCDF4.Variable'>
 int32 centaur_key(no_of_markers)
 unlimited dimensions: 
 current shape = (7,)
 filling on, default _FillValue of -2147483647 used,
 'points_of_tetraeders': <class 'netCDF4._netCDF4.Variable'>
 int32 points_of_tetraeders(no_of_tetraeders, points_per_tetraeder)
 unlimited dimensions: 
 current shape = (9222, 4)
 filling on, default _FillValue of -2147483647 used,
 'points_of_prisms': <class 'netCDF4._netCDF4.Variable'>
 int32 points_of_prisms(no_of_prisms, points_per_prism)
 unlimited dimensions: 
 current shape = (762, 6)
 filling on, default _FillValue of -2147483647 used,
 'points_of_pyramids': <class 'netCDF4._netCDF4.Variable'>
 int32 points_of_pyramids(no_of_pyramids, points_per_pyramid)
 unlimited dimensions: 
 current shape = (180, 5)
 filling on, default

In [65]:
b_o_s = mdata.variables["boundarymarker_of_surfaces"][:]
p_o_s = mdata.variables["points_of_surfacetriangles"][:]
p_o_q = mdata.variables["points_of_surfacequadrilaterals"][:]
xc  = mdata.variables["points_xc"]
yc  = mdata.variables["points_yc"]
zc  = mdata.variables["points_zc"]
p_o_s

masked_array(
  data=[[  2,  60,  72],
        [ 74,  24,   5],
        [ 73,  75,  74],
        ...,
        [726, 845, 699],
        [764, 710, 709],
        [710, 810, 709]],
  mask=False,
  fill_value=999999,
  dtype=int32)

In [27]:
indices_of_marker = np.where((b_o_s == 7) | (b_o_s == 1))
indices_of_marker

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34, 652, 653, 654, 655,
        656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668,
        669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679]),)

In [71]:
nan_pt = pt.zeros(size=(len(p_o_s),1))
nan_pt[add_pt==0] = float('nan')
# add nan to every triangle as 4th element so we can join the tensors
filled_p_o_s = pt.cat((pt.tensor(p_o_s, dtype=int), add_pt),1)
# join the tensors of tris and quads together
points_of_tris_and_quads = pt.cat((filled_p_o_s, pt.tensor(p_o_q)))

tensor([[  2.,  60.,  72.,  nan],
        [ 74.,  24.,   5.,  nan],
        [ 73.,  75.,  74.,  nan],
        ...,
        [162.,  14.,  15., 163.],
        [163.,  15.,  16., 164.],
        [164.,  16.,   2.,  72.]])

In [ ]:
# For Triangles only
indices_of_marker = np.where(b_o_s == 1)
global_id_of_marker_points = np.unique(p_o_s[indices_of_marker].flatten())
xs = xc[global_id_of_marker_points]
ys = yc[global_id_of_marker_points]
zs = zc[global_id_of_marker_points]
global_id_of_marker_points

In [ ]:
sfile = loader._file_name(times[0])
sdata = Dataset(sfile)

In [ ]:
gid = sdata.variables["global_id"][:]
cp = sdata.variables["cp"][:]
gid

In [ ]:
indices = np.where(np.in1d(gid, global_id_of_marker_points))[0]
indices

In [ ]:
cp = sdata.variables["cp"][:][np.where(np.in1d(gid,global_id_of_marker_points))[0]]
xtest = sdata.variables["x"][:][np.where(np.in1d(gid,global_id_of_marker_points))[0]]
ytest = sdata.variables["y"][:][np.where(np.in1d(gid,global_id_of_marker_points))[0]]
fig, ax = plt.subplots()
ax.scatter(xs,ys, marker='^')
ax.scatter(xtest,ytest, marker='v')
plt.show()

# Prototyping

In [ ]:
import numpy as np
import torch as pt
from netCDF4 import Dataset
from os.path import join

In [ ]:
mfile = join(loader._para.path, loader._para.config["primary_grid"])
mdata = Dataset(mfile)

In [ ]:
points_x = mdata.variables["points_xc"]
points_y = mdata.variables["points_yc"]
points_z = mdata.variables["points_zc"]
points_tri = mdata.variables["points_of_surfacetriangles"]
points_quad = mdata.variables["points_of_surfacequadrilaterals"]
marker_list = mdata.variables["boundarymarker_of_surfaces"]

In [ ]:
"""
# Assuming you have the following lists
points_x = np.array([0.5, 0.2, 0.8, 0.6, 0.4, 0.7, 0.3, 0.9])
points_y = np.array([0.1, 0.4, 0.6, 0.9, 0.2, 0.5, 0.8, 0.3])
points_z = np.array([0.7, 0.3, 0.9, 0.4, 0.8, 0.5, 0.2, 0.6])
points_tri = [[0, 1, 2], [3, 4, 5], [6, 7, 0], [1, 3, 5]]
points_quad = [[0, 1, 2, 3], [4, 5, 6, 7]]
marker_list = [200, 200, 200, 400, 500, 200]
"""

# Combine x, y, z coordinates into a single array
points_xyz = np.column_stack((points_x, points_y, points_z))

# Create a dictionary to map unique points to their indices and markers
unique_points = {}
unique_markers = {}

# Iterate over triangle list and marker list
for triangle, marker in zip(points_tri, marker_list[:len(points_tri)]):
    points_triangle = points_xyz[triangle]
    for point in points_triangle:
        point_key = tuple(np.append(point,marker))
        if point_key not in unique_points:
            unique_points[point_key] = len(unique_points)
            unique_markers[point_key] = marker

# Iterate over quad list and marker list
for quad, marker in zip(points_quad, marker_list[len(points_tri):]):
    points_quad = points_xyz[quad]
    for point in points_quad:
        point_key = tuple(np.append(point,marker))
        if point_key not in unique_points:
            unique_points[point_key] = len(unique_points)
            unique_markers[point_key] = marker

# Create a new array to store unique points and markers
unique_array = np.zeros((len(unique_points), 4))  # Assuming 3D points and 1 marker value
unique_array[:, :] = np.array(list(unique_points.keys()))

# Remove duplicate points with same markers
_, unique_indices = np.unique(unique_array[:, :], axis=0, return_index=True)
unique_array = unique_array[unique_indices]

print(unique_array.shape)

# Faster version

In [ ]:
mfile = join(loader._para.path, loader._para.config["primary_grid"])
mdata = Dataset(mfile)
points_x = mdata.variables["points_xc"][:]
points_y = mdata.variables["points_yc"][:]
points_z = mdata.variables["points_zc"][:]
points_tri = mdata.variables["points_of_surfacetriangles"][:]
points_quad = mdata.variables["points_of_surfacequadrilaterals"][:]
marker_list = mdata.variables["boundarymarker_of_surfaces"][:]

In [ ]:
import numpy as np

# Assuming you have the following lists
points_x = np.array([0.5, 0.2, 0.8, 0.6, 0.4, 0.7, 0.3, 0.9])
points_y = np.array([0.1, 0.4, 0.6, 0.9, 0.2, 0.5, 0.8, 0.3])
points_z = np.array([0.7, 0.3, 0.9, 0.4, 0.8, 0.5, 0.2, 0.6])
points_tri = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 0], [1, 3, 5]])
points_quad = np.array([[0, 1, 2, 3], [4, 5, 6, 7]])
marker_list = np.array([200, 200, 200, 400, 500, 200])

# Combine x, y, z coordinates into a single array
points_xyz = np.column_stack((points_x, points_y, points_z))

# Create a new array to store combined indices
num_triangles = len(points_tri)
num_quads = len(points_quad)
total_triangles = num_triangles + num_quads
points_indices = np.full((total_triangles, 4), -1, dtype=int)
points_indices[:num_triangles, :3] = points_tri
points_indices[num_triangles:, :] = points_quad

# Combine marker_list for triangle and quad
combined_marker_list = np.concatenate((marker_list[:num_triangles], marker_list[num_triangles:]))

# Create a structured array to store unique points and markers
dtype = [('x', float), ('y', float), ('z', float), ('marker', int)]
unique_points = []
unique_markers = []

# Iterate over indices and marker_list simultaneously
for indices, marker in zip(points_indices, combined_marker_list):
    indices = indices[indices != -1]  # Remove -1 values
    points = points_xyz[indices]
    unique_points.append(points)
    unique_markers.append(np.full(len(points), marker))

unique_points = np.concatenate(unique_points)
unique_markers = np.concatenate(unique_markers)

# Remove duplicate points with same markers
_, unique_indices = np.unique(np.column_stack((unique_points, unique_markers)), return_index=True)
unique_array = np.empty(len(unique_indices), dtype=dtype)
unique_array['x'] = unique_points[unique_indices, 0]
unique_array['y'] = unique_points[unique_indices, 1]
unique_array['z'] = unique_points[unique_indices, 2]
unique_array['marker'] = unique_markers[unique_indices]

print(unique_array)